#Importar archivos y librerías

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/content/trainPrepocesado.csv')
y_train = train['RENDIMIENTO_GLOBAL']
x_train = train.drop(columns=['RENDIMIENTO_GLOBAL'])
train.head()

,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,0.074380,0.00,0.857143,0.25,0.500000,1.0,0.555556,1.000000,0.0,1.000000
1,0.024793,0.25,0.571429,0.00,0.500000,0.0,0.666667,0.555556,0.0,0.333333
2,0.115702,0.00,0.571429,1.00,0.500000,1.0,0.444444,0.444444,0.0,0.333333
3,0.008264,0.00,0.714286,0.00,0.666667,1.0,0.000000,0.444444,0.0,0.000000
4,0.041322,0.00,0.571429,0.75,0.500000,1.0,0.222222,0.222222,0.0,0.666667


In [ ]:
y_train.head()

,RENDIMIENTO_GLOBAL
0,1.000000
1,0.333333
2,0.333333
3,0.000000
4,0.666667


In [ ]:
x_test = pd.read_csv('/content/testPrepocesado.csv')
x_test.drop(columns=['Unnamed: 0'], inplace=True)
x_test.head()

,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0.066116,0.25,0.142857,0.25,0.500000,1.0,0.666667,0.222222,1.0
1,0.008264,0.00,0.571429,0.75,0.333333,1.0,0.444444,0.666667,0.0
2,0.000000,0.00,0.428571,0.00,0.500000,1.0,0.333333,0.444444,0.0
3,0.033058,0.25,0.428571,0.75,0.166667,0.0,0.111111,0.111111,0.0
4,0.008264,0.25,0.571429,0.50,0.333333,1.0,0.444444,0.444444,0.0


In [ ]:
y_test = pd.read_csv('/content/submission_examplePrepocesado.csv')
y_test.drop(columns=['ID'], inplace=True)
y_test.head()

,RENDIMIENTO_GLOBAL
0,0.666667
1,0.666667
2,0.000000
3,0.333333
4,0.666667


#Sobremuestreo y modelo KNeighborsClassifier

Se importa las librerías responsable del modelo y desde el modelo hasta los preprocesamiento de los datos antes de incluirlos dentro del modelo.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

Se usa los metoso de sobremuestreo para emparejar equilibrar las clases con el fin de obtener un mayor conjunto de datos y equilibrados.

In [ ]:
y_train = y_train.astype(str)
y_test = y_test.astype(str)
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [ ]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [ ]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.4322575862742852
Classification Report:
                    precision    recall  f1-score   support

               0.0       0.41      0.56      0.48     74539
0.3333333333333333       0.43      0.47      0.45     74176
0.6666666666666666       0.44      0.39      0.41     74084
               1.0       0.47      0.31      0.37     73987

          accuracy                           0.43    296786
         macro avg       0.44      0.43      0.43    296786
      weighted avg       0.44      0.43      0.43    296786



Con esta funcion buscamos la predicion corresponda a un las ingresadas por el creador de la competencia y legibles para el kaggle.

In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns=['RENDIMIENTO_GLOBAL'])
mapa = {
    '0.0': 'bajo',
    '0.3333333333333333': 'medio-bajo',
    '0.6666666666666666': 'medio-alto',
    '1.0': 'alto'
}
y_pred_df['RENDIMIENTO_GLOBAL'] = y_pred_df['RENDIMIENTO_GLOBAL'].map(mapa)
y_pred_df =  pd.concat([pd.read_csv('/content/submission_examplePrepocesado.csv')['ID'], y_pred_df], axis=1)
y_pred_df.to_csv('sln.csv', index=False)